## Notebook to compute the precision and recall metrics for the training and test sets of the models

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sn
import matplotlib.pyplot as plt

### Training set

In [26]:
aux_thesis = pd.read_pickle("../data/cna_brca_train_0.8_threshold_0.6_chrX.pkl")

In [27]:
aux_thesis.dtypes[aux_thesis.dtypes !='float64']

tcga_id             object
Ciriello_subtype    object
dtype: object

In [28]:
aux_thesis.shape

(817, 386)

In [39]:
import ast
res_train = pd.read_csv("../results/fully_con_ieee/rna+miRNA+cna/50_hidden_emb_tcga_classifier_dropout_0.5_cv.csv")

In [40]:
res_train

,Unnamed: 0,Fold,accuracy,other_metrics,mean_accuracy,hidden_1,batch_size,epochs_vae,learning_rate,dropout_input
0,0,1,0.878049,"{'Basal': {'precision': 0.9642857142857143, 'r...",0.86518,50,50,100,0.001,0.5
1,1,2,0.871166,"{'Basal': {'precision': 1.0, 'recall': 1.0, 'f...",0.86518,50,50,100,0.001,0.5
2,2,3,0.901841,"{'Basal': {'precision': 1.0, 'recall': 1.0, 'f...",0.86518,50,50,100,0.001,0.5
3,3,4,0.822086,"{'Basal': {'precision': 0.92, 'recall': 0.8518...",0.86518,50,50,100,0.001,0.5
4,4,5,0.852761,"{'Basal': {'precision': 0.896551724137931, 're...",0.86518,50,50,100,0.001,0.5


In [41]:
res_train["accuracy"]

0    0.878049
1    0.871166
2    0.901841
3    0.822086
4    0.852761
Name: accuracy, dtype: float64

In [42]:
res_train["mean_accuracy"]

0    0.86518
1    0.86518
2    0.86518
3    0.86518
4    0.86518
Name: mean_accuracy, dtype: float64

In [43]:
res_train["other_metrics"]

0    {'Basal': {'precision': 0.9642857142857143, 'r...
1    {'Basal': {'precision': 1.0, 'recall': 1.0, 'f...
2    {'Basal': {'precision': 1.0, 'recall': 1.0, 'f...
3    {'Basal': {'precision': 0.92, 'recall': 0.8518...
4    {'Basal': {'precision': 0.896551724137931, 're...
Name: other_metrics, dtype: object

In [44]:
from statistics import stdev
subtypes = ["Basal", "Her2", "LumA", "LumB", "Normal"]
weights_train=[135,65,415,176,25] # Ones for all things miRNA
#weights_train=[136,65,415,176,25] # Ones for rna and cna
mean_precisions = []
mean_recalls = []

for i in range(0,5):
    dict_aux = ast.literal_eval(res_train["other_metrics"].values[i])
    arr_pre = []
    arr_rec = []
    for sub in subtypes:
        arr_pre.append(dict_aux[sub]['precision'])
        arr_rec.append(dict_aux[sub]['recall'])
    mean_precisions.append(np.average(arr_pre, weights=weights_train))
    mean_recalls.append(np.average(arr_rec, weights=weights_train))
    
print(mean_precisions)
print('{}+-{}'.format(np.mean(mean_precisions), stdev(mean_precisions)))
print("----------------")
print(mean_recalls)
print('{}+-{}'.format(np.mean(mean_recalls), stdev(mean_recalls)))

[0.8525380269000511, 0.8691169282962452, 0.9015605173693408, 0.8311528097518809, 0.8447293978443304]
0.8598195360323697+-0.027070578435362133
----------------
[0.8782679738562092, 0.8710434173669468, 0.9016806722689076, 0.8219887955182072, 0.852626050420168]
0.8651213818860878+-0.029837218424254266


In [45]:
print("Average accuracy: {}+-{}".format(np.mean(res_train["accuracy"].values), stdev(res_train["accuracy"].values)))

Average accuracy: 0.8651803135871887+-0.029819307041289378


In [1]:
# no miRNA
(43*1+16*0.982+131*0.942+32*0.966+14*0.982)/(43+16+131+32+14)

0.9609067796610168

In [23]:
# miRNA
(36*0.9945+15*0.985+128*0.923+26*0.953+14*1)/(36+15+128+26+14)

0.9474840182648402

### Test set

In [125]:
res_test = pd.read_csv("../results/miRNA/CVAE/100_hidden_20_emb/tcga_classifier_dropout_0.8_in_0.2_hidden_rec_loss_binary_crossentropy_classifier_frozen_False_cv_other_metrics.csv")

In [126]:
res_test['other_metrics']

0    {'Basal': {'precision': 0.896551724137931, 're...
1    {'Basal': {'precision': 0.9629629629629629, 'r...
2    {'Basal': {'precision': 0.8620689655172413, 'r...
3    {'Basal': {'precision': 0.96, 'recall': 0.8888...
4    {'Basal': {'precision': 0.7878787878787878, 'r...
Name: other_metrics, dtype: object

In [127]:
res_test

,Unnamed: 0,Fold,accuracy,other_metrics,mean_accuracy,intermediate_dim,latent_dim,batch_size,epochs_cvae,learning_rate,dropout_input,dropout_hidden,dropout_decoder,freeze_weights,classifier_use_z,classifier_loss,reconstruction_loss
0,0,1,0.810976,"{'Basal': {'precision': 0.896551724137931, 're...",0.769557,100,20,200,100,0.01,0.8,0.2,True,False,False,categorical_crossentropy,binary_crossentropy
1,1,2,0.779141,"{'Basal': {'precision': 0.9629629629629629, 'r...",0.769557,100,20,200,100,0.01,0.8,0.2,True,False,False,categorical_crossentropy,binary_crossentropy
2,2,3,0.711656,"{'Basal': {'precision': 0.8620689655172413, 'r...",0.769557,100,20,200,100,0.01,0.8,0.2,True,False,False,categorical_crossentropy,binary_crossentropy
3,3,4,0.779141,"{'Basal': {'precision': 0.96, 'recall': 0.8888...",0.769557,100,20,200,100,0.01,0.8,0.2,True,False,False,categorical_crossentropy,binary_crossentropy
4,4,5,0.766871,"{'Basal': {'precision': 0.7878787878787878, 'r...",0.769557,100,20,200,100,0.01,0.8,0.2,True,False,False,categorical_crossentropy,binary_crossentropy


In [128]:
subtypes = ["Basal", "Her2", "LumA", "LumB", "Normal"]
weights_test=[36,15,128,26,14]
mean_precisions = []
mean_recalls = []

dict_aux = ast.literal_eval(res_test['other_metrics'][0])

print("Accuracy {}".format(res_test["accuracy"].values))
print("Precision {}".format(dict_aux['weighted avg']['precision']))
print("Recall {}".format(dict_aux['weighted avg']['recall']))

Accuracy [0.81097561 0.77914113 0.71165645 0.77914113 0.76687115]
Precision 0.7956924118972507
Recall 0.8109756097560976


In [129]:
print("Average accuracy: {}".format(np.mean(res_test["accuracy"].values)))

Average accuracy: 0.7695570945739746
